In [1]:
import cv2
import numpy as np
import pickle
import pandas as pd
from ultralytics import YOLO

# Instruction STEP 7:
# Load data from the pickle file
with open("ROI_Cam4_0&1", "rb") as f:
    data = pickle.load(f)
polylines, area_list = data['polylines'], data['area_list']

# Read class labels from file
with open("our_class_list.txt", "r") as my_file:
    class_list = my_file.read().split("\n")

# Initialize YOLO models
custom_model = YOLO('last.pt')
pretrained_model = YOLO('yolov8n.pt')

# Instruction STEP 8:
cap = cv2.VideoCapture("Edge_Crossing_1_Cam4_0&1.avi")

bag_last_position = {}
abandoned_bag_duration_threshold = 200
alert_duration = 30  # Duration of the alert message
count=0
while True:
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset video to beginning
        continue
    count += 1
    if count % 3 != 0:
        continue

    frame = cv2.resize(frame, (640, 640))

    # Predict objects using custom and pre-trained YOLO models
    custom_results = custom_model.predict(frame)
    pretrained_results = pretrained_model.predict(frame)

    custom_detections = custom_results[0].boxes.data.cpu().numpy().astype("float")
    pretrained_detections = pretrained_results[0].boxes.data.cpu().numpy().astype("float")

    list1 = []
    bag_centers = []
    train_detected = False
    show_alert = False  # Flag to control the alert message display

    # Iterate over the detected objects from custom model
    for detection in custom_detections:
        x1, y1, x2, y2, conf, cls_id = detection
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cx = int(x1 + 0.15 * (x2 - x1))  # Adjust center point x-coordinate
        cy = int(y2)  # Adjust center point y-coordinate
        cls_name = class_list[int(cls_id)]

        # Check if the object is a person and its center point is within any ROI polygon
        if 'Humans' in cls_name:
            for polyline in polylines:
                if cv2.pointPolygonTest(polyline, (cx, cy), False) >= 0:
                    if not train_detected:
                        list1.append((x1, y1, x2, y2))  # Append bounding box coordinates
                        show_alert = True  # Set the flag to show the alert message
                    break

        if 'Baggage' in cls_name:
            bag_center = (cx, cy)
            bag_centers.append(bag_center)
            cv2.circle(frame, (int(cx), int(cy)), 2, (0, 255, 255), -1)

    # Iterate over the detected objects from pre-trained model
    for detection in pretrained_detections:
        x1, y1, x2, y2, conf, cls_id = detection
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cx = int(x1 + 0.5 * (x2 - x1))  # Adjust center point x-coordinate
        cy = int(y2)  # Adjust center point y-coordinate
        cls_name = pretrained_model.names[int(cls_id)]

        # Check if a train is detected
        if 'train' in cls_name:
            train_detected = True
            break

    # If a train is detected, skip appending person and human bounding boxes
    if train_detected:
        list1 = []

    # Draw bounding boxes around people within the ROI
    for x1, y1, x2, y2 in list1:
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # Display person count
    person_count = len(list1)
    cv2.putText(frame, f'Person Count in restricted area: {person_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Display the alert message if a person is detected within the ROI and train is not detected
    if show_alert and not train_detected:
        cv2.putText(frame, 'Alert', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        alert_duration -= 1  # Decrement the alert duration counter
        if alert_duration == 0:
            show_alert = False  # Reset the flag after 30 frames
            alert_duration = 30  # Reset the alert duration counter

    # Code for detecting abandoned bags
    for bag_center in bag_centers:
        if bag_center in bag_last_position:
            distance = np.sqrt((bag_center[0] - bag_last_position[bag_center]['position'][0]) ** 2 +
                               (bag_center[1] - bag_last_position[bag_center]['position'][1]) ** 2)
            if distance <= 10:  # Adjust motion threshold as needed
                bag_last_position[bag_center]['duration'] += 1
            else:
                bag_last_position[bag_center] = {'position': bag_center, 'duration': 0}
        else:
            bag_last_position[bag_center] = {'position': bag_center, 'duration': 0}

        if bag_last_position[bag_center]['duration'] >= abandoned_bag_duration_threshold:
            cv2.putText(frame, 'Abandoned Bag!', bag_center, cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 1)

    # Show the frame
    cv2.imshow('FRAME', frame)

    # Check for key press to exit
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()

C:\Users\Manas\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 640x640 1 Humans, 219.2ms
Speed: 2.9ms preprocess, 219.2ms inference, 1327.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 97.5ms
Speed: 2.0ms preprocess, 97.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Humans, 188.3ms
Speed: 2.9ms preprocess, 188.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 82.0ms
Speed: 2.0ms preprocess, 82.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Humans, 164.0ms
Speed: 2.1ms preprocess, 164.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 79.4ms
Speed: 2.0ms preprocess, 79.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Humans, 163.7ms
Speed: 2.0ms preprocess, 163.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 78.9ms
Speed: 2.0ms preprocess, 78.9ms inference, 1.0ms po